Проект - предсказать, является ли сообщение spam или ham

Импорт нужных библиотек

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import string
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

Загрузка Датаcета и его изучение

In [ ]:
data = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')
data.head()

In [ ]:
data.info()

удаляем Ненужные 

In [ ]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis= 1, inplace = True)

Поменяем названия для Более удобного использование

In [ ]:
data.rename(columns={'v1': 'label', 'v2': 'text'}, inplace=True)
data.head()

Смотрим

In [ ]:
data.groupby('label').describe()


Для лучшего понимания данных и повышения точности с нашей моделью Добавим столбец с Количеством символов в Сообщении

In [ ]:
data['length'] = data['text'].apply(len)
data.head()

In [ ]:
data.length.describe()


визуализация

In [ ]:
data['length'].plot(bins=100, kind='hist') 

In [ ]:
data.hist(column='length', by='label', bins=50, figsize=(15,5))


создадим функцию, которая запомнит наш набор данных и удалит ненужные знаки препинания и стоп-слова.

In [ ]:
def text_process(mess):
    
    nopunc = [char for char in mess if char not in string.punctuation]

    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

разделение набора данных для тренировки

In [ ]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, label_train, label_test = \
train_test_split(data['text'], data['label'], test_size=0.3)

print(len(msg_train), len(msg_test), len(msg_train) + len(msg_test))

Размер теста составляет 30% от всего набора данных 1672 сообщения из общего числа 5572, а для обучение - остальное 3900 из 5572
Теперь мы можем использовать практически любые алгоритмы классификации.

Возьмем Logistic regression

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)), 
    ('tfidf', TfidfTransformer()),  
    ('classifier', LogisticRegression()),  
])
pipeline.fit(msg_train,label_train)
predictions=pipeline.predict(msg_test)
print(classification_report(predictions,label_test))
lrscore=accuracy_score(predictions,label_test)
print(lrscore)

Для сравнение возьмем Naive Bayes

In [ ]:
pipeline1 = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)), 
    ('tfidf', TfidfTransformer()), 
    ('classifier', MultinomialNB()), 
])
pipeline1.fit(msg_train,label_train)
predictions1 = pipeline1.predict(msg_test)
print(classification_report(predictions1,label_test))
nbscore=accuracy_score(predictions1,label_test)
print(nbscore)

Посмотрим результаты для сравнение

In [ ]:
results=[lrscore,nbscore]
n=['Logis. R.','Naiva Bayse']
ndf=pd.DataFrame(n)
rdf=pd.DataFrame(results)
rdf[1]=n
print('Accuracy')
rdf

и видем что Naiva Bayse больше подходит чем Logistic regression